<a href="https://colab.research.google.com/github/nmpacheco/colab/blob/fastai/testing_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare env

In [0]:
!pip3 install fastai==0.7.0
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install "torchtext==0.2.3"

  Using cached https://files.pythonhosted.org/packages/5b/a5/e8b50b55b1abac9f1e3346c4242f1e42a82d368a8442cbd50c532922f6c4/torch-0.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [0]:
!apt-get -qq install -y libsm6 libxext6

In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from os import path, makedirs
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.torch_imports import *
from fastai.io import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *

#from torch import * 
import torch.nn as nn

from google.colab import drive

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
print('{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag()))

cp36-cp36m


In [0]:
mypath = 'data/mnist/'

In [0]:
makedirs(mypath, exist_ok=True)

In [7]:
!pwd

/content


In [12]:
print('cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu')

cpu


In [0]:
torch.cuda.is_available()
torch.backends.cudnn.enabled

In [0]:
URL='http://deeplearning.net/data/mnist/'
FILENAME='mnist.pkl.gz'

def load_mnist(filename):
    return pickle.load(gzip.open(filename, 'rb'), encoding='latin-1')

In [9]:
get_data(URL+FILENAME, path+FILENAME)
((x, y), (x_valid, y_valid), _) = load_mnist(path+FILENAME)

mnist.pkl.gz: 16.2MB [00:03, 4.63MB/s]                            


cpu


True